# 패키지 로드

In [ ]:
%pip install torch datasets
%pip install transformers accelerate peft trl

In [1]:
import torch
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer

# 데이터 가져오기

In [2]:
import json

def load_train_dataset_from_json(path: str):
    with open(path, "r", encoding="utf-8-sig") as f:
        data = json.load(f)
        return data

# 사용 예시
train_dataset = load_train_dataset_from_json("qwen3_company_train_dataset_combined.json")


## 데이터 구조 확인

In [3]:
# 데이터셋 길이
len(train_dataset)

2450

In [4]:
# 데이터셋 구조
train_dataset[:2]

[{'messages': [{'role': 'system',
    'content': '\n당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.\n다음 지침을 따르세요:\n1. 기존의 말투는 잊고 정중하고 사무적인 어조로 답변해야 하세요.\n2. 대화 내역의 말투도 참고하지 말고 무조건 정중하고 사문적인 어조로 답변하세요\n3. 사실에 기반한 정보를 사용하세요.\n4. 사용자의 질문에 대한 답변을 문서에서 찾을 수 없을 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.\n5. 사용자가 문서에 대한 질문이 아닌, "안녕"과 같은 일상적인 질문을 한다면 해당 내용에 대해서 적절히 답변해주세요.\n6. 답변이 너무 길지 않게 하세요.\n7. 사용자가 편하게 반말로 물어보더라도, 반드시 정중하고 사무적인 어조로 답변해야 합니다.\n8. 사용자는 frontend(프론트엔드)팀에 속한 팀원입니다. 질문이 들어오면 반드시 frontend_search 툴을 호출하여 문서를 검색하세요.\n'},
   {'role': 'user',
    'content': "빌드/배포 가이드의 '1.1 환경 설정' 파트에서 Node.js 설치에 대한 구체적인 버전 요구사항은 무엇인가요?"},
   {'role': 'assistant',
    'content': '<tool_call>{"name": "frontend_search", "arguments": {"keyword": "빌드/배포 가이드의 \'1.1 환경 설정\' 파트에서 Node.js 설치에 대한 구체적인 버전 요구사항은 무엇인가요?"}}</tool_call>'},
   {'role': 'user',
    'content': '<tool_response>검색 결과:\n-----\n<!-- 회사: 코드노바 | 대상: 사원(프론트엔드) | 작성일: 2025-08-29 -->\n# 빌드/배포 가이드\n분류: frontend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-0

## 리스트 형태에서 Dataset 객체로 변경

In [5]:
print(type(train_dataset))

train_dataset = Dataset.from_list(train_dataset)

print(type(train_dataset))

<class 'list'>
<class 'datasets.arrow_dataset.Dataset'>


# 모델 로드 및 템플릿 적용

In [7]:
# 허깅페이스 모델 ID
model_id = "Qwen/Qwen3-8B"

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype=torch.bfloat16,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
# 템플릿 적용 확인
text = tokenizer.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system

당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.
다음 지침을 따르세요:
1. 기존의 말투는 잊고 정중하고 사무적인 어조로 답변해야 하세요.
2. 대화 내역의 말투도 참고하지 말고 무조건 정중하고 사문적인 어조로 답변하세요
3. 사실에 기반한 정보를 사용하세요.
4. 사용자의 질문에 대한 답변을 문서에서 찾을 수 없을 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.
5. 사용자가 문서에 대한 질문이 아닌, "안녕"과 같은 일상적인 질문을 한다면 해당 내용에 대해서 적절히 답변해주세요.
6. 답변이 너무 길지 않게 하세요.
7. 사용자가 편하게 반말로 물어보더라도, 반드시 정중하고 사무적인 어조로 답변해야 합니다.
8. 사용자는 frontend(프론트엔드)팀에 속한 팀원입니다. 질문이 들어오면 반드시 frontend_search 툴을 호출하여 문서를 검색하세요.
<|im_end|>
<|im_start|>user
빌드/배포 가이드의 '1.1 환경 설정' 파트에서 Node.js 설치에 대한 구체적인 버전 요구사항은 무엇인가요?<|im_end|>
<|im_start|>assistant
<tool_call>{"name": "frontend_search", "arguments": {"keyword": "빌드/배포 가이드의 '1.1 환경 설정' 파트에서 Node.js 설치에 대한 구체적인 버전 요구사항은 무엇인가요?"}}</tool_call><|im_end|>
<|im_start|>user
<tool_response>검색 결과:
-----
<!-- 회사: 코드노바 | 대상: 사원(프론트엔드) | 작성일: 2025-08-29 -->
# 빌드/배포 가이드
분류: frontend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29

---

## 1. 빌드 준비

### 1.1 환경 설정
- **Node.js 설치**: 최신 LTS 버전을 설치합니다.
- **패키지 매니

# LoRA와 SFTConfig 설정

In [9]:
peft_config = LoraConfig(
        lora_alpha=32,
        lora_dropout=0.1,
        r=8,
        bias="none",
        target_modules=["q_proj", "v_proj"],
        task_type="CAUSAL_LM",
)

In [10]:
args = SFTConfig(
    output_dir="qwen3-8b-informal-formal",          # 저장될 디렉토리와 저장소 ID
    num_train_epochs=3,                      # 학습할 총 에포크 수
    per_device_train_batch_size=4,           # GPU당 배치 크기
    gradient_accumulation_steps=2,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=[]
)

# 학습 중 전처리 함수: collate_fn

In [11]:
def collate_fn(batch):
    new_batch = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for example in batch:
        # messages의 각 내용에서 개행문자 제거
        clean_messages = []
        for message in example["messages"]:
            clean_message = {
                "role": message["role"],
                "content": message["content"]
            }
            clean_messages.append(clean_message)

        # 깨끗해진 메시지로 템플릿 적용
        text = tokenizer.apply_chat_template(
            clean_messages,
            tokenize=False,
            add_generation_prompt=False
        ).strip()

        # 텍스트를 토큰화
        tokenized = tokenizer(
            text,
            truncation=True,
            max_length=8192,
            padding=False,
            return_tensors=None,
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        # 레이블 초기화
        labels = [-100] * len(input_ids)

        # assistant 응답 부분 찾기
        im_start = "<|im_start|>"
        im_end = "<|im_end|>"
        assistant = "assistant"

        # 토큰 ID 가져오기
        im_start_tokens = tokenizer.encode(im_start, add_special_tokens=False)
        im_end_tokens = tokenizer.encode(im_end, add_special_tokens=False)
        assistant_tokens = tokenizer.encode(assistant, add_special_tokens=False)

        i = 0
        while i < len(input_ids):
            # <|im_start|>assistant 찾기
            if (i + len(im_start_tokens) <= len(input_ids) and
                input_ids[i:i+len(im_start_tokens)] == im_start_tokens):

                # assistant 토큰 찾기
                assistant_pos = i + len(im_start_tokens)
                if (assistant_pos + len(assistant_tokens) <= len(input_ids) and
                    input_ids[assistant_pos:assistant_pos+len(assistant_tokens)] == assistant_tokens):

                    # assistant 응답의 시작 위치로 이동
                    current_pos = assistant_pos + len(assistant_tokens)

                    # <|im_end|>를 찾을 때까지 레이블 설정
                    while current_pos < len(input_ids):
                        if (current_pos + len(im_end_tokens) <= len(input_ids) and
                            input_ids[current_pos:current_pos+len(im_end_tokens)] == im_end_tokens):
                            # <|im_end|> 토큰도 레이블에 포함
                            for j in range(len(im_end_tokens)):
                                labels[current_pos + j] = input_ids[current_pos + j]
                            break
                        labels[current_pos] = input_ids[current_pos]
                        current_pos += 1

                    i = current_pos

            i += 1

        new_batch["input_ids"].append(input_ids)
        new_batch["attention_mask"].append(attention_mask)
        new_batch["labels"].append(labels)

    # 패딩 적용
    max_length = max(len(ids) for ids in new_batch["input_ids"])

    for i in range(len(new_batch["input_ids"])):
        padding_length = max_length - len(new_batch["input_ids"][i])

        new_batch["input_ids"][i].extend([tokenizer.pad_token_id] * padding_length)
        new_batch["attention_mask"][i].extend([0] * padding_length)
        new_batch["labels"][i].extend([-100] * padding_length)

    # 텐서로 변환
    for k, v in new_batch.items():
        new_batch[k] = torch.tensor(v)

    return new_batch

# 학습

In [12]:
# collate_fn 테스트 (배치 크기 1로)
example = train_dataset[0]
batch = collate_fn([example])

print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("레이블 형태:", batch["labels"].shape)


처리된 배치 데이터:
입력 ID 형태: torch.Size([1, 2781])
어텐션 마스크 형태: torch.Size([1, 2781])
레이블 형태: torch.Size([1, 2781])


In [13]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
)

In [14]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,0.911300
20,0.518100
30,0.399500
40,0.389600
50,0.339700
60,0.357300
70,0.329900
80,0.336100
90,0.321100
100,0.306000


# 파인 튜닝 모델 테스트

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

In [16]:
class QwenChatbot:
    def __init__(self,
                 model_name="Qwen/Qwen3-8B",
                 lora_path=None,
                 system_message="You are a helpful AI assistant."):

        # 토크나이저 및 기본 모델 로드
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        base_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto"
        )

        # LoRA 적용
        if lora_path:
            print(f"LoRA 어댑터 로드: {lora_path}")
            self.model = PeftModel.from_pretrained(base_model, lora_path)
        else:
            self.model = base_model

        self.history = []

        # 시스템 메시지 초기화
        if system_message:
            self.history.append({"role": "system", "content": system_message})

    def generate_response(self, user_input, max_new_tokens=1024):
        # 대화 기록 + 사용자 입력
        messages = self.history + [{"role": "user", "content": user_input}]

        # Chat 템플릿 적용
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # 모델 입력 변환
        inputs = self.tokenizer(text, return_tensors="pt").to(self.model.device)

        # 응답 생성
        response_ids = self.model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
        )[0][len(inputs.input_ids[0]):].tolist()

        response = self.tokenizer.decode(response_ids, skip_special_tokens=True)

        # 히스토리 업데이트
        self.history.append({"role": "user", "content": user_input})
        self.history.append({"role": "assistant", "content": response})

        return response

In [17]:
permission = 'cto'

tone_instruction = f"정중하고 사무적인 어조"

tool_prompt = f"사용자는 {permission}로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다. 질문이 들어오면 반드시 {permission}_search 툴을 호출하여 문서를 검색하세요."

system_message = f"""

당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.
다음 지침을 따르세요:
1. 기존의 말투는 잊고 정중하고 {tone_instruction}로 답변해야 하세요.
2. 대화 내역의 말투도 참고하지 말고 무조건 {tone_instruction}로 답변하세요
3. 사실에 기반한 정보를 사용하세요.
4. 사용자의 질문에 대한 답변을 문서에서 찾을 수 없을 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.
5. 사용자가 문서에 대한 질문이 아닌, "안녕"과 같은 일상적인 질문을 한다면 해당 내용에 대해서 적절히 답변해주세요.
6. 답변이 너무 길지 않게 하세요.
7. 사용자가 편하게 반말로 물어보더라도, 반드시 {tone_instruction}로 답변해야 합니다.
8. {tool_prompt}
"""

In [20]:
# Example Usage
if __name__ == "__main__":
    chatbot = QwenChatbot(
        model_name="Qwen/Qwen3-8B",
        lora_path="qwen3-8b-informal-formal/checkpoint-921",
        system_message=system_message
    )

    # First input
    user_input_1 = "서비스 아키텍처 문서에서 API 서버의 검증 포인트는 무엇인가요?"
    print(f"User: {user_input_1}")
    response_1 = chatbot.generate_response(user_input_1)
    print(f"Bot: {response_1}")
    print("----------------------")

    # Second input
    user_input_2 = "API 서버의 검증 포인트에 대한 구체적인 테스트 방법은 무엇인가요?"
    print(f"User: {user_input_2}")
    response_2 = chatbot.generate_response(user_input_2)
    print(f"Bot: {response_2}")
    print("----------------------")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


LoRA 어댑터 로드: qwen3-8b-informal-formal/checkpoint-921
User: 서비스 아키텍처 문서에서 API 서버의 검증 포인트는 무엇인가요?
Bot: <tool_call>{"name": "cto_search", "arguments": {"keyword": "서비스 아키텍처 문서에서 API 서버의 검증 포인트는 무엇인가요?"}}</tool_call>
----------------------
User: API 서버의 검증 포인트에 대한 구체적인 테스트 방법은 무엇인가요?
Bot: <tool_call>{"name": "cto_search", "arguments": {"keyword": "API 서버의 검증 포인트에 대한 구체적인 테스트 방법은 무엇인가요?"}}</tool_call>
----------------------


In [21]:
permission = 'backend'

tone_instruction = f"정중하고 사무적인 어조"

tool_prompt = f"사용자는 {permission}(백엔드)팀에 속한 팀원입니다. 질문이 들어오면 반드시 {permission}_search 툴을 호출하여 문서를 검색하세요."

system_message = f"""

당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.
다음 지침을 따르세요:
1. 기존의 말투는 잊고 {tone_instruction}로 답변해야 하세요.
2. 대화 내역의 말투도 참고하지 말고 무조건 {tone_instruction}로 답변하세요
3. 사실에 기반한 정보를 사용하세요.
4. 사용자의 질문에 대한 답변을 문서에서 찾을 수 없을 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.
5. 사용자가 문서에 대한 질문이 아닌, "안녕"과 같은 일상적인 질문을 한다면 해당 내용에 대해서 적절히 답변해주세요.
6. 답변이 너무 길지 않게 하세요.
7. 사용자가 편하게 반말로 물어보더라도, 반드시 {tone_instruction}로 답변해야 합니다.
8. {tool_prompt}
"""

In [ ]:
# Example Usage
if __name__ == "__main__":
    chatbot = QwenChatbot(
        model_name="Qwen/Qwen3-8B",
        lora_path="qwen3-8b-informal-formal/checkpoint-921",
        system_message=system_message
    )

    # First input
    user_input_1 = "백엔드팀 주간 업무 계획서의 다음 주 업무 계획 논의는 어떤 방식으로 진행되나요?"
    print(f"User: {user_input_1}")
    response_1 = chatbot.generate_response(user_input_1)
    print(f"Bot: {response_1}")
    print("----------------------")

    # Second input
    user_input_2 = "기능 개발 로드맵 업데이트에서 각 기능의 개발 일정은 어떻게 설정되고 있나요?"
    print(f"User: {user_input_2}")
    response_2 = chatbot.generate_response(user_input_2)
    print(f"Bot: {response_2}")
    print("----------------------")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

LoRA 어댑터 로드: qwen3-8b-informal-formal/checkpoint-921
User: 백엔드팀 주간 업무 계획서의 다음 주 업무 계획 논의는 어떤 방식으로 진행되나요?


In [20]:
permission = 'frontend'

tone_instruction = f"정중하고 사무적인 어조"

tool_prompt = f"사용자는 {permission}(프론트엔드)팀에 속한 팀원입니다. 질문이 들어오면 반드시 {permission}_search 툴을 호출하여 문서를 검색하세요."

system_message = f"""

당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.
다음 지침을 따르세요:
1. 기존의 말투는 잊고 {tone_instruction}로 답변해야 하세요.
2. 대화 내역의 말투도 참고하지 말고 무조건 {tone_instruction}로 답변하세요
3. 사실에 기반한 정보를 사용하세요.
4. 사용자의 질문에 대한 답변을 문서에서 찾을 수 없을 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.
5. 사용자가 문서에 대한 질문이 아닌, "안녕"과 같은 일상적인 질문을 한다면 해당 내용에 대해서 적절히 답변해주세요.
6. 답변이 너무 길지 않게 하세요.
7. 사용자가 편하게 반말로 물어보더라도, 반드시 {tone_instruction}로 답변해야 합니다.
8. {tool_prompt}
"""

In [21]:
# Example Usage
if __name__ == "__main__":
    chatbot = QwenChatbot(
        model_name="Qwen/Qwen3-8B",
        lora_path="qwen3-8b-informal-formal/checkpoint-1839",
        system_message=system_message
    )

    # First input
    user_input_1 = "타이포그래피 섹션에서 폰트 크기 설정에 대한 구체적인 기준은 무엇인가요?"
    print(f"User: {user_input_1}")
    response_1 = chatbot.generate_response(user_input_1)
    print(f"Bot: {response_1}")
    print("----------------------")

    # Second input
    user_input_2 = "협업 프로세스 문서의 초기 회의 섹션에서 요구사항이 명확히 정의되었는지 검증하는 방법은 무엇인가요?"
    print(f"User: {user_input_2}")
    response_2 = chatbot.generate_response(user_input_2)
    print(f"Bot: {response_2}")
    print("----------------------")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

LoRA 어댑터 로드: qwen3-8b-informal-formal/checkpoint-1839
User: 타이포그래피 섹션에서 폰트 크기 설정에 대한 구체적인 기준은 무엇인가요?
Bot: <tool_call>{"name": "frontend_search", "arguments": {"keyword": "타이포그래피 섹션에서 폰트 크기 설정에 대한 구체적인 기준은 무엇인가요?"}}</tool_call>
----------------------
User: 협업 프로세스 문서의 초기 회의 섹션에서 요구사항이 명확히 정의되었는지 검증하는 방법은 무엇인가요?
Bot: <tool_call>{"name": "frontend_search", "arguments": {"keyword": "협업 프로세스 문서의 초기 회의 섹션에서 요구사항이 명확히 정의되었는지 검증하는 방법은 무엇인가요?"}}</tool_call>
----------------------


In [22]:
permission = 'data_ai'

tone_instruction = f"정중하고 사무적인 어조"

tool_prompt = f"사용자는 Data AI(데이터 AI)팀에 속한 팀원입니다. 질문이 들어오면 반드시 {permission}_search 툴을 호출하여 문서를 검색하세요."

system_message = f"""

당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.
다음 지침을 따르세요:
1. 기존의 말투는 잊고 {tone_instruction}로 답변해야 하세요.
2. 대화 내역의 말투도 참고하지 말고 무조건 {tone_instruction}로 답변하세요
3. 사실에 기반한 정보를 사용하세요.
4. 사용자의 질문에 대한 답변을 문서에서 찾을 수 없을 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.
5. 사용자가 문서에 대한 질문이 아닌, "안녕"과 같은 일상적인 질문을 한다면 해당 내용에 대해서 적절히 답변해주세요.
6. 답변이 너무 길지 않게 하세요.
7. 사용자가 편하게 반말로 물어보더라도, 반드시 {tone_instruction}로 답변해야 합니다.
8. {tool_prompt}
"""

In [23]:
# Example Usage
if __name__ == "__main__":
    chatbot = QwenChatbot(
        model_name="Qwen/Qwen3-8B",
        lora_path="qwen3-8b-informal-formal/checkpoint-1839",
        system_message=system_message
    )

    # First input
    user_input_1 = "정기적인 보안 감사 및 취약점 점검은 어떤 주기로 실시하나요?"
    print(f"User: {user_input_1}")
    response_1 = chatbot.generate_response(user_input_1)
    print(f"Bot: {response_1}")
    print("----------------------")

    # Second input
    user_input_2 = "데이터/AI팀 회의록(정기회의) #2에서 데이터 품질 개선 방안 논의 시 강조된 필요성은 무엇인가요?"
    print(f"User: {user_input_2}")
    response_2 = chatbot.generate_response(user_input_2)
    print(f"Bot: {response_2}")
    print("----------------------")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

LoRA 어댑터 로드: qwen3-8b-informal-formal/checkpoint-1839
User: 정기적인 보안 감사 및 취약점 점검은 어떤 주기로 실시하나요?
Bot: <tool_call>{"name": "data_ai_search", "arguments": {"keyword": "정기적인 보안 감사 및 취약점 점검은 어떤 주기로 실시하나요?"}}</tool_call>
----------------------
User: 데이터/AI팀 회의록(정기회의) #2에서 데이터 품질 개선 방안 논의 시 강조된 필요성은 무엇인가요?
Bot: <tool_call>{"name": "data_ai_search", "arguments": {"keyword": "데이터/AI팀 회의록(정기회의) #2에서 데이터 품질 개선 방안 논의 시 강조된 필요성은 무엇인가요?"}}</tool_call>
----------------------


# 모델 병합

In [5]:
import os

USER_NAME = "SKN14-Final-1Team"
MODEL_NAME = 'qwen3-8b-informal-formal-merged-09-17'
REPO_ID = f"{USER_NAME}/{MODEL_NAME}"

In [8]:
from peft import AutoPeftModelForCausalLM

peft_model_id = "qwen3-8b-informal-formal/checkpoint-921"
fine_tuned_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map="auto", dtype=torch.float16)

# LoRA → base weight에 병합
merged_model = fine_tuned_model.merge_and_unload()

# 허브에 업로드
merged_model.push_to_hub(REPO_ID)
tokenizer.push_to_hub(REPO_ID)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

No files have been modified since last commit. Skipping to prevent empty commit.


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/SKN14-Final-1Team/qwen3-8b-informal-formal-merged-09-17/commit/b5eb20b6120a492a570d1c512056af111906aa82', commit_message='Upload tokenizer', commit_description='', oid='b5eb20b6120a492a570d1c512056af111906aa82', pr_url=None, repo_url=RepoUrl('https://huggingface.co/SKN14-Final-1Team/qwen3-8b-informal-formal-merged-09-17', endpoint='https://huggingface.co', repo_type='model', repo_id='SKN14-Final-1Team/qwen3-8b-informal-formal-merged-09-17'), pr_revision=None, pr_num=None)

## 허깅페이스 파인튜닝 모델 불러와서 테스트 

In [9]:
class QwenChatbot:
    def __init__(self,
                 model_name="Qwen/Qwen3-8B",
                 lora_path=None,
                 system_message="You are a helpful AI assistant."):

        # 토크나이저 및 기본 모델 로드
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        base_model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto"
        )

        # LoRA 적용
        if lora_path:
            print(f"LoRA 어댑터 로드: {lora_path}")
            self.model = PeftModel.from_pretrained(base_model, lora_path)
        else:
            self.model = base_model

        self.history = []

        # 시스템 메시지 초기화
        if system_message:
            self.history.append({"role": "system", "content": system_message})

    def generate_response(self, user_input, max_new_tokens=1024):
        # 대화 기록 + 사용자 입력
        messages = self.history + [{"role": "user", "content": user_input}]

        # Chat 템플릿 적용
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        # 모델 입력 변환
        inputs = self.tokenizer(text, return_tensors="pt").to(self.model.device)

        # 응답 생성
        response_ids = self.model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
        )[0][len(inputs.input_ids[0]):].tolist()

        response = self.tokenizer.decode(response_ids, skip_special_tokens=True)

        # 히스토리 업데이트
        self.history.append({"role": "user", "content": user_input})
        self.history.append({"role": "assistant", "content": response})

        return response

In [10]:
permission = 'data_ai'

tone_instruction = f"정중하고 사무적인 어조"

tool_prompt = f"사용자는 Data AI(데이터 AI)팀에 속한 팀원입니다. 질문이 들어오면 반드시 {permission}_search 툴을 호출하여 문서를 검색하세요."

system_message = f"""

당신은 사내 지식을 활용하여 사용자의 질문에 정확하고 유용한 답변을 제공하는 한국인 AI 비서입니다.
다음 지침을 따르세요:
1. 기존의 말투는 잊고 {tone_instruction}로 답변해야 하세요.
2. 대화 내역의 말투도 참고하지 말고 무조건 {tone_instruction}로 답변하세요
3. 사실에 기반한 정보를 사용하세요.
4. 사용자의 질문에 대한 답변을 문서에서 찾을 수 없을 경우, "잘 모르겠습니다"라고 솔직하게 말하세요.
5. 사용자가 문서에 대한 질문이 아닌, "안녕"과 같은 일상적인 질문을 한다면 해당 내용에 대해서 적절히 답변해주세요.
6. 답변이 너무 길지 않게 하세요.
7. 사용자가 편하게 반말로 물어보더라도, 반드시 {tone_instruction}로 답변해야 합니다.
8. {tool_prompt}
"""

In [11]:
# Example Usage
if __name__ == "__main__":
    chatbot = QwenChatbot(
        model_name="SKN14-Final-1Team/qwen3-8b-informal-formal-merged-09-17",
        lora_path=None,
        system_message=system_message
    )

    # First input
    user_input_1 = "서비스 아키텍처 문서에서 API 서버의 검증 포인트는 무엇인가요?"
    print(f"User: {user_input_1}")
    response_1 = chatbot.generate_response(user_input_1)
    print(f"Bot: {response_1}")
    print("----------------------")

    # Second input
    user_input_2 = "API 서버의 검증 포인트에 대한 구체적인 테스트 방법은 무엇인가요?"
    print(f"User: {user_input_2}")
    response_2 = chatbot.generate_response(user_input_2)
    print(f"Bot: {response_2}")
    print("----------------------")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


User: 서비스 아키텍처 문서에서 API 서버의 검증 포인트는 무엇인가요?
Bot: <tool_call>{"name": "data_ai_search", "arguments": {"keyword": "서비스 아키텍처 문서에서 API 서버의 검증 포인트는 무엇인가요?"}}</tool_call>
----------------------
User: API 서버의 검증 포인트에 대한 구체적인 테스트 방법은 무엇인가요?
Bot: <tool_call>{"name": "data_ai_search", "arguments": {"keyword": "API 서버의 검증 포인트에 대한 구체적인 테스트 방법은 무엇인가요?"}}</tool_call>
----------------------


## 일상 질문도 tool call 하는 상태.. 아마 답변에는 문제 없을듯함.

In [5]:
# Example Usage
if __name__ == "__main__":
    chatbot = QwenChatbot(
        model_name="SKN14-Final-1Team/qwen3-8b-informal-formal-merged-09-17",
        lora_path=None,
        system_message=system_message
    )

    # First input
    user_input_1 = "안녕?"
    print(f"User: {user_input_1}")
    response_1 = chatbot.generate_response(user_input_1)
    print(f"Bot: {response_1}")
    print("----------------------")

    # Second input
    user_input_2 = "너는 누구야?"
    print(f"User: {user_input_2}")
    response_2 = chatbot.generate_response(user_input_2)
    print(f"Bot: {response_2}")
    print("----------------------")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

User: 안녕?
Bot: <tool_call>{"name": "data_ai_search", "arguments": {"keyword": "안녕?"}}</tool_call>
----------------------
User: 너는 누구야?
Bot: <tool_call>{"name": "data_ai_search", "arguments": {"keyword": "너는 누구야?"}}</tool_call>
----------------------
